<a href="https://colab.research.google.com/github/Shristi0124/Weather-prediction/blob/main/Weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests #This libarary help us to fech data from API
import pandas as pd # for handaling and analysing data
import numpy as np #for numerical operation
from sklearn.model_selection import train_test_split # to split data into trainig and testing sets
from sklearn.preprocessing import LabelEncoder # to convert cetogorical data into numericals values
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor # model for classification and regression tasks
from sklearn.metrics import mean_squared_error #to measure the accuracy of our prediction
from datetime import datetime , timedelta # to handle data and time
import pytz

In [ ]:
API_KEY= '71774b38b5f0c49b1b75d5fa195b63a9'  # REPLACE WITH YOUR ACUTAL API KEY
BASE_URL = 'https://api.openweathermap.org/data/2.5/'# Base url for making API requests

Fetch Current Weather  data

In [ ]:
def get_current_weather(city):
  url= f"{BASE_URL}/weather?q={city}&appid={API_KEY}&units=metric" # construct the API request URL
  response = requests.get(url)  # send the get request to  API
  data = response.json()
  if response.status_code != 200:
      print(f"Error fetching weather data for {city}: {data.get('message', 'Unknown error')}")
      return None
  return{
      'city': data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like':round(data['main']['feels_like']),
      'temp_min': round(data['main']['temp_min']),
      'temp_max': round(data['main']['temp_max']),
      'humidity': round(data['main']['humidity']),
      'description': data['weather'][0]['description'],
      'country': data['sys']['country'],
      'wind_gust_dir':data['wind']['deg'],
      'pressure' : data['main']['pressure'],
      'Wind_Gust_Speed':data['wind']['speed']

  }

2. Read Historical Data

In [ ]:
def read_historical_data(filename):
  df= pd .read_csv(filename) #load csv file into dataFrame
  df = df.dropna() # remove rows with missing values
  df = df.drop_duplicates()
  return df


3. prepare data for training

In [ ]:
def prepare_data(data):
  le = LabelEncoder()  #  create a LableEncoder instance

  # Define all possible compass points
  compass_points_list = [
      "Ν", "NNE", "NE", "ΕΝΕ", "Ε", "ESE", "SE", "SSE", "S", "SSW", "SW",
      "WSW", "W", "WNW", "NW", "NNW"
  ]

  # Combine historical data wind directions with all compass points for fitting the encoder
  all_wind_dirs = pd.concat([data['WindGustDir'], pd.Series(compass_points_list)]).unique()

  le.fit(all_wind_dirs)

  data['WindGustDir'] =le.transform(data['WindGustDir'])

  # Fit the LabelEncoder on the possible values for 'RainTomorrow'
  le_rain = LabelEncoder()
  le_rain.fit(['Yes', 'No'])
  data['RainTomorrow'] = le_rain.transform(data['RainTomorrow'])


  #define the feature variable and target variables
  X = data[['MinTemp','MaxTemp' , 'WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']] #feature variable
  y = data['RainTomorrow'] # traget variable


  return X,y,le_rain # return feture variable , target variable and the lable encoder for rain

4. Train Rain predictin Model

In [ ]:
def train_rain_model(X,y):
  X_train ,X_test ,y_train , y_test = train_test_split(X,y,test_size=0.2, random_state= 42)
  model = RandomForestClassifier(n_estimators=100, random_state= 42)
  model.fit(X_train,y_train)     #train the model

  y_pred = model.predict(X_test)   #to make prediction on test set

  print("Mean Squared Error for Rain Model")

  print(mean_squared_error(y_test,y_pred))

  return model

5. prepare regrassion data

In [ ]:
def prepare_regression_data(data,feature):
  X,y = [], []  # initialize list for feature and target value

  if len(data) < 2:  # Check if there are enough data points
      return X, y

  for i  in range(len(data)-1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1,1)
  y= np.array(y)


  return X,y

In [ ]:
def train_regression_model(X,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X,y)

  return model

Peredict future

In [ ]:
def predict_future(model , current_value):
  predictions = [current_value]
  #return model # This line was causing the function to return the model prematurely

  for i in range (5):
    # Use the correct method name 'predict' instead of 'prediction'
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])


  return predictions[1:]

Weather Anaylsis Funtion

In [ ]:
from time import timezone
import pytz
from datetime import datetime , timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

def weather_view():
  city = input('Enter  any city name :')
  current_weather = get_current_weather(city)

  if current_weather is None:
      return

  #load historical data

  historical_data = read_historical_data('/content/weather.csv')

  #prepare and train the rain prediction model

  X,y,le = prepare_data(historical_data)

  rain_model = train_rain_model(X,y)
  wind_deg = current_weather['wind_gust_dir']% 360
  compass_points =[

      ("Ν", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
("ΕΝΕ", 56.25, 78.75), ("Ε", 78.75, 101.25), ("ESE", 101.25, 123.75),
("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25), ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
("NNW", 326.25, 348.75)
  ]
  compass_direction =next(point for point,start, end in compass_points if start <= wind_deg  <end)

  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in  le.classes_ else -1

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir':compass_direction_encoded,
      'WindGustSpeed':current_weather['Wind_Gust_Speed'],
      'Humidity':current_weather['humidity'],
      'Pressure':current_weather['pressure'],
      'Temp':current_weather['current_temp'],

  }

  current_df =pd.DataFrame([current_data])


  #rain prediction

  rain_prediction = rain_model.predict(current_df)[0]


  #prepare regression model for temperature and humidity

  X_temp , y_temp = prepare_regression_data(historical_data, 'Temp')

  X_hum, y_hum = prepare_regression_data(historical_data ,'Humidity')

  # Check if there is enough data for regression before training and predicting
  if X_temp.size > 0 and X_hum.size > 0:
    temp_model = train_regression_model(X_temp,y_temp)
    hum_model = train_regression_model(X_hum, y_hum)


    #predict future temperature and humidity


    future_temp = predict_future(temp_model,current_weather['temp_min'])



    future_humidity = predict_future(hum_model,current_weather['humidity'])

  else:
      future_temp = []
      future_humidity = []
      print("Insufficient historical data to train regression models for future predictions.")


  #prepare time for future predictions

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hours = next_hour.replace(minute=0, second = 0 , microsecond = 0 )

  future_times = [(next_hour +timedelta(hours=i)).strftime("%H:00") for i in range(5)]


   # Display results
  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Description: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")
  print(f"\nFuture Temperature Predictions:")

  for time , temp in zip(future_times,future_temp):


    print(f"{time}: {round(temp ,1)}°C")
  print("\nFuture Humidity Predictions:")

  for time , humidity in zip(future_times,future_humidity):
     print(f"{time}: {round(humidity ,1)}°C")
weather_view()

Enter  any city name :dehradun
Mean Squared Error for Rain Model
0.1506849315068493
City: dehradun, IN
Current Temperature: 23°C
Feels Like: 24°C
Minimum Temperature: 23°C
Maximum Temperature: 23°C
Humidity: 98%
Weather Description: moderate rain
Rain Prediction: Yes

Future Temperature Predictions:
03:00: 18.9°C
04:00: 23.1°C
05:00: 18.0°C
06:00: 20.6°C
07:00: 21.3°C

Future Humidity Predictions:
03:00: 67.2°C
04:00: 76.3°C
05:00: 61.9°C
06:00: 56.5°C
07:00: 40.7°C
